<a href="https://colab.research.google.com/github/aryamtos/augmentation-processing-ddsm/blob/master/InceptionV3_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
from os import listdir 
from os.path import isdir 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras import layers 
from tensorflow.keras import models 
from tensorflow.keras import optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def select_image(filename):
    
    image = Image.open(filename) #abrindo aquivo
    
    image = image.convert('RGB')
    image = image.resize((150,150)) #redimensiona a imagem
    
    return np.asarray(image)  #converte num numpy array

# Carregando duas classes
# Essa função vai pegar os diretórios e organizar os labels (etiquetar as imagens)
def load_class(diretorio, classe, imagens, labels):
    #Iterando arquivos
    
    for filename in listdir(diretorio):
        
        path = diretorio + filename #monta o endereço da imagem 
        
        try:
            imagens.append(select_image(path))
            labels.append(classe)
        except:
            print('Erro ao ler imagem{}'.format(path))
    return imagens, labels

# Selecionando Dataset

# A função select_data_set() vai selecionar todo o dataset e verificar quais são os subdiretórios com a subdir.

def select_data_set(diretorio):
    
    imagens = list()
    labels = list()
    
    for subdir in listdir(diretorio):
        path = diretorio + subdir + '/'
        
        if not isdir(path):
            continue
        imagens, labels = load_class(path, subdir, imagens, labels)
    
    return imagens, labels


nodulos_dataset = '/content/drive/MyDrive/train/'

imagens,labels = select_data_set(nodulos_dataset)

imagens = np.array(imagens)/255 #convertendo de lista para array
labels = np.array(labels) # conversão de lista para array

# Tratando os labels
lb =LabelBinarizer()
labels = lb.fit_transform(labels) 
# Categorizar os dados
labels = to_categorical(labels)

In [ ]:
batch_size = 50 #processamento a cada 54 imagens
input_shape = (150, 150, 3) #tamanho da imagem
random_state = 42 # serve para reproduzir a "experiência"
alpha = 1e-5 # taxa de aprendizado
epoch = 100

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [ ]:

NAME = "inception20_v3"
alpha = 1e-5
tensorboard = TensorBoard(log_dir = "/content/drive/My Drive/{}".format(NAME))

file_path="/content/drive/My Drive/inceptionT20_v3.best.hdf5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="val_acc", mode="max", patience=15)

lr_reduce = ReduceLROnPlateau(monitor='acc', factor=0.1, min_delta=alpha, patience= 5, verbose= 1)

callbacks_list = [checkpoint, lr_reduce,tensorboard]

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(imagens, labels, test_size = 0.2, stratify = labels, random_state=random_state)


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True)

train_datagen.fit(trainX)

data_aug = train_datagen.flow(trainX, trainY, batch_size= batch_size)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-12-22 03:04:28--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.20.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   161MB/s    in 0.5s    

2020-12-22 03:04:29 (161 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
#local_weights_file = 'imagenet'
pre_trained_model = InceptionV3(
    input_shape=(150,150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [ ]:

from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers


In [ ]:
model = Sequential()
model.add(pre_trained_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
#x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation

# Add a dropout rate of 0.2



# Add a final sigmoid layer for classification
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(2, activation='softmax')(x)
# Configure and compile the model

# optimizer=RMSprop(lr=0.0001)

model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(data_aug, steps_per_epoch=len(trainX)//batch_size, validation_data=(testX, testY),
                              validation_steps= len(testX)//batch_size,callbacks=callbacks_list, epochs = 100,
                              verbose=2)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
39/39 - 15s - loss: 0.6503 - acc: 0.6120 - val_loss: 0.5906 - val_acc: 0.7245

Epoch 00001: val_acc improved from -inf to 0.72449, saving model to /content/drive/My Drive/inceptionT20_v3.best.hdf5
Epoch 2/100
39/39 - 10s - loss: 0.5666 - acc: 0.7063 - val_loss: 0.5407 - val_acc: 0.7816

Epoch 00002: val_acc improved from 0.72449 to 0.78163, saving model to /content/drive/My Drive/inceptionT20_v3.best.hdf5
Epoch 3/100
39/39 - 10s - loss: 0.5379 - acc: 0.7263 - val_loss: 0.5030 - val_acc: 0.8000

Epoch 00003: val_acc improved from 0.78163 to 0.80000, saving model to /content/drive/My Drive/inceptionT20_v3.best.hdf5
Epoch 4/100
39/39 - 10s - loss: 0.5168 - acc: 0.7472 - val_loss: 0.5081 - val_acc: 0.7816

Epoch 00004: val_acc did not improve from 0.80000
Epoch 5/100
39/39 - 10s - loss: 0.5102 - acc: 0.7389 - val_loss: 0.5120 - val_acc: 0.7327

Epoch 00005: val_acc did not improve from 0.80000
Epoch 6/100
39/39 - 10s - loss: 0.5298 - acc: 0.7347 - val_loss: 0.4882 - val_acc: 0.

In [ ]:
from tensorflow.keras.optimizers import SGD

unfreeze = False

# Unfreeze all models after "mixed6"
for layer in pre_trained_model.layers:
  if unfreeze:
    layer.trainable = True
  if layer.name == 'mixed6':
    unfreeze = True

# As an optimizer, here we will use SGD 
# with a very low learning rate (0.00001)
model.compile(loss='binary_crossentropy',
              optimizer=SGD(
                  lr=0.00001, 
                  momentum=0.9),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(data_aug, steps_per_epoch=100, validation_data=(testX, testY),
                              validation_steps= 50,callbacks=callbacks_list, epochs = 50,
                              verbose=2)



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
100/100 - 15s - loss: 0.4405 - acc: 0.7706 - val_loss: 0.3785 - val_acc: 0.8429

Epoch 00001: val_acc did not improve from 0.85306


In [ ]:
model.save('/content/drive/MyDrive/inception18_v3.h5')